In [8]:
pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


In [ ]:
tiers=["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
divisions = ["I", "II", "III", "IV"]
pages=["1","2"]

In [ ]:
#https://developer.riotgames.com/apis#summoner-v4/GET_getBySummonerName
#https://developer.riotgames.com/apis#match-v5/GET_getMatchIdsByPUUID

In [ ]:
import csv
import requests
import time  # Import the time module

def fetch_and_save_ranked_players(tiers, divisions, pages):
    api_key = "RGAPI-9700eac4-1e3c-4da8-92d2-d2168387e08f"  # Use your actual API key
    base_url = "https://euw1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
    headers = {
        "X-Riot-Token": api_key
    }
    
    file_path = '../DataProcessing/TestData/rankedPlayers.csv'
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['tier', 'division', 'summonerName'])  # CSV Header

        for tier in tiers:
            for division in divisions:
                for page in pages:
                    url = base_url.format(tier=tier, division=division, page=page)
                    response = requests.get(url, headers=headers)

                    # Wait for 2 seconds before making the next request
                    time.sleep(2)

                    if response.status_code == 200:
                        players = response.json()
                        for player in players:
                            if player['wins'] + player['losses'] > 20:
                                writer.writerow([player['tier'], player['rank'], player['summonerName']])
                    else:
                        print(f"Error: {response.status_code}, {response.text}")
# Note: Before running, replace 'Your_API_Key_Here' with your actual API key and ensure the file path for CSV is correct as per your environment.


In [ ]:
#ONLY RUN IF NEW DATA REQUIRED
#fetch_and_save_ranked_players(tiers, divisions, pages)

In [ ]:
import csv

def find_duplicate_summoner_names_with_rows(file_path):
    summoner_occurrences = {}  # Tracks summoner names and their row numbers
    duplicates_info = []  # Stores information about duplicates

    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row_number, row in enumerate(reader, start=1):  # Start counting rows from 1
            summoner_name = row['summonerName']
            if summoner_name in summoner_occurrences:
                # If the summoner name is already encountered, add the current row as a duplicate
                summoner_occurrences[summoner_name].append(row_number)
            else:
                # Otherwise, initialize the list with the current row number
                summoner_occurrences[summoner_name] = [row_number]

    # Filter out summoner names with more than one occurrence and prepare duplicate info
    for summoner_name, row_numbers in summoner_occurrences.items():
        if len(row_numbers) > 1:
            duplicates_info.append((summoner_name, row_numbers))

    return duplicates_info

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
#duplicates = find_duplicate_summoner_names_with_rows(file_path)

# Example output
#for summoner_name, rows in duplicates:
#    print(f"Duplicate: {summoner_name} found in rows: {rows}")


In [ ]:
import csv

def remove_duplicate_summoner_names(file_path):
    processed_summoners = set()  # To track unique summonerNames
    unique_rows = []  # To store rows after removing duplicates

    # Read the file and filter out duplicate summonerNames
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            if row['summonerName'] not in processed_summoners:
                processed_summoners.add(row['summonerName'])
                unique_rows.append(row)

    # Write the unique rows back to the file
    with open(file_path, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['tier', 'division', 'summonerName']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(unique_rows)

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
#remove_duplicate_summoner_names(file_path)


In [25]:
import os
import csv

def split_csv_by_gold_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "PLATINUM":
                file_name = "PLATINUM_players.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayers.csv'

# Call the function
split_csv_by_gold_tier(source_file_path)


In [24]:
def split_csv_by_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of CSV writers for each tier
    writers = {}
    files = {}
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            
            tier = row['tier']
            file_name = f"{tier}_players.csv"
            file_path = os.path.join(base_folder, file_name)
            
            # Check if we already have a writer for this tier, if not create it
            if tier not in writers:
                files[tier] = open(file_path, mode='w', newline='', encoding='utf-8')
                writers[tier] = csv.DictWriter(files[tier], fieldnames=reader.fieldnames)
                writers[tier].writeheader()
            
            # Write the row to the appropriate file
            writers[tier].writerow(row)
    
    # Close all the files we've opened
    for file in files.values():
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayers.csv'

# Call the function
split_csv_by_tier(source_file_path)


In [26]:
def get_summoner_names_from_csv(file_path):
    summoner_names = []
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            summoner_names.append(row['summonerName'])
    
    return summoner_names

# Specify the path to your CSV file
file_path = '../DataProcessing/TestData/PLATINUM_players.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)


['Lil Pump', '3XpEnDaBlE', 'Wickly', 'MrScene', 'drombom', '2JZ SINGLE TURBO', 'Benwailord2', 'YA ISEIKYA', 'TRASHDRAVENXD', 'Mr  Puff', 'PIBAB DE KEBZA', 'zozoricardo', 'Lana Del Ahrey', 'Nogitsune101', 'Death13n2', 'Matthicc', 'aleix4444', 'Yuuriko', 'KaNeKi       0', 'GCB G2 SurbEnzo', 'SOLO SINGED', 'Jajacubu', 'JugoJugovice', 'StarGuardianAdc', 'MAZĘ', 'HitMeBaby1', 'XSoulh', 'Dešreliu Uogiene', 'Dolboyob', 'F9Smôkey', 'Titeuf V2', 'Petrichoria', 'Sisoo', 'KATABLOOD', 'Not Genji', 'Revolution 909', 'VKJ', 'Mudda Simon', 'carlos larios', 'S K I N E U R E', 'Milchryze', 'R6Ezio', 'jsuis p2 nero', 'PFP Paulux', 'CordeWow', 'Hawk280', 'Nevia', 'YasTraining666', 'GoldChronos', 'Teddybaer2901', 'luucaasssss', 'Xulaxe12', 'LethalFoxdiver', 'Mo4ning Myrtle', 'biao jiu ya', 'sina3es', 'ProductionsZ', 'Uraxee', 'Zazlost', 'Windie the Pooh', 'Emil Schöer', 'Hyper boner', 'MET4 ABUSER', 'Trippy Mane', 'Shactas', 'bocata pollo GOZ', 'Cha0sVal3ri0', 'UdoVonNebenan', 'Lσst Hope', '0tt0o', 'Nοst'

In [27]:
import requests
import time
import csv
import os
#FINDS PUUID FROM SUMMONER NAME
def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-b8c6b8bc-cae0-444d-b593-c5795065be0d"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    summoner_to_puuid = {}

    start_time = time.time()
    last_update_time = start_time
    total_names = len(summoner_names)
    names_processed = 0

    # Fetch puuid for each summoner name
    for name in summoner_names:
        url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            summoner_to_puuid[name] = data['puuid']
        else:
            print(f"Error fetching data for {name}: {response.status_code}")
        names_processed += 1
        current_time = time.time()

        # Check if 1 minute has passed for progress update
        if current_time - last_update_time >= 60:
            print(f"Progress: {names_processed}/{total_names} summoner names processed.")
            last_update_time = current_time

        time.sleep(1.5)  # Delay to comply with rate limit
    
    # Update progress after finishing all requests
    print(f"Progress: {names_processed}/{total_names} summoner names processed. Update complete.")

    # Now, replace summoner names with puuid in the CSV
    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    # Replace original file with the updated temp file
    os.replace(temp_file_path, file_path)


# Call the function
update_summoner_names_with_puuid(file_path, summoner_names)



Progress: 38/1014 summoner names processed.
Progress: 75/1014 summoner names processed.
Progress: 112/1014 summoner names processed.
Progress: 150/1014 summoner names processed.
Progress: 188/1014 summoner names processed.
Progress: 225/1014 summoner names processed.
Progress: 262/1014 summoner names processed.
Progress: 299/1014 summoner names processed.
Progress: 337/1014 summoner names processed.
Progress: 375/1014 summoner names processed.
Progress: 413/1014 summoner names processed.
Progress: 451/1014 summoner names processed.
Progress: 489/1014 summoner names processed.
Progress: 527/1014 summoner names processed.
Progress: 565/1014 summoner names processed.
Progress: 603/1014 summoner names processed.
Progress: 641/1014 summoner names processed.
Progress: 678/1014 summoner names processed.
Progress: 716/1014 summoner names processed.
Progress: 753/1014 summoner names processed.
Progress: 791/1014 summoner names processed.
Progress: 828/1014 summoner names processed.
Progress: 86

In [12]:
import asyncio
import aiohttp
import csv
import os
from aiohttp import ClientSession
#FINDS PUUID FROM SUMMONER NAME v2
async def fetch_puuid(session, url, headers, semaphore, summoner_to_puuid, name):
    async with semaphore:
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                data = await response.json()
                summoner_to_puuid[name] = data['puuid']
            else:
                print(f"Error fetching data for {name}: {response.status}")

async def rate_limited_requester(summoner_names, headers, rate_limit=30):
    tasks = []
    summoner_to_puuid = {}

    semaphore = asyncio.Semaphore(rate_limit)

    async with ClientSession() as session:
        for name in summoner_names:
            url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
            task = asyncio.create_task(fetch_puuid(session, url, headers, semaphore, summoner_to_puuid, name))
            tasks.append(task)

            if len(tasks) % rate_limit == 0:
                await asyncio.sleep(1)

        await asyncio.gather(*tasks)

    return summoner_to_puuid

async def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }

    summoner_to_puuid = await rate_limited_requester(summoner_names, headers)

    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    os.replace(temp_file_path, file_path)

# Execution block adapted for Jupyter Notebook
loop = asyncio.get_event_loop()

source_csv_path = "../DataProcessing/TestData/GOLD_players.csv"


if loop.is_running():
    task = loop.create_task(update_summoner_names_with_puuid(source_csv_path, summoner_names))
else:
    loop.run_until_complete(update_summoner_names_with_puuid(source_csv_path, summoner_names))


In [28]:
import requests
import csv
import time
import os
#Finds matches from puuid
def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "RGAPI-b8c6b8bc-cae0-444d-b593-c5795065be0d"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage
source_csv_path = '../DataProcessing/TestData/PLATINUM_players.csv'
target_csv_path = '../DataProcessing/TestData/PLATINUM_GAMES.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)


Progress: 38 summoners processed.
Progress: 75 summoners processed.
Progress: 112 summoners processed.
Progress: 149 summoners processed.
Progress: 186 summoners processed.
Progress: 223 summoners processed.
Progress: 260 summoners processed.
Progress: 297 summoners processed.
Progress: 334 summoners processed.
Progress: 371 summoners processed.
Progress: 408 summoners processed.
Progress: 445 summoners processed.
Progress: 482 summoners processed.
Progress: 519 summoners processed.
Progress: 556 summoners processed.
Progress: 593 summoners processed.
Progress: 630 summoners processed.
Progress: 667 summoners processed.
Progress: 704 summoners processed.
Progress: 741 summoners processed.
Progress: 778 summoners processed.
Progress: 815 summoners processed.
Progress: 852 summoners processed.
Progress: 889 summoners processed.
Progress: 926 summoners processed.
Progress: 963 summoners processed.
Progress: 1000 summoners processed.
Final Progress: 1014 summoners processed. Task complete.

In [19]:
import asyncio
import aiohttp
import csv
import os
from aiohttp import ClientSession
import datetime

async def fetch_match_ids(session, url, csv_writer, sem, progress):
    async with sem:
        async with session.get(url) as response:
            if response.status == 200:
                match_ids = await response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
                progress['count'] += 1  # Increment the progress count for each successful fetch
            else:
                print(f"Error: {response.status}")

async def run(source_csv_path, target_csv_path, api_key):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    sem = asyncio.Semaphore(30)  # Control concurrency to 30 requests per second
    progress = {'count': 0, 'last_update': datetime.datetime.now()}  # Initialize progress tracking

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    # Open the target CSV file outside of the async context
    with open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Writing header for match IDs

        async with ClientSession(headers=headers) as session:
            tasks = []
            with open(source_csv_path, mode='r', encoding='utf-8') as infile:
                csv_reader = csv.DictReader(infile)
                for row in csv_reader:
                    puuid = row['summonerName']
                    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
                    task = asyncio.create_task(fetch_match_ids(session, url, csv_writer, sem, progress))
                    tasks.append(task)
                    
                    # Check and print progress approximately every minute
                    if (datetime.datetime.now() - progress['last_update']).seconds >= 60:
                        print(f"Progress: {progress['count']} summonerNames processed at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                        progress['last_update'] = datetime.datetime.now()

                    if len(tasks) >= 30:
                        await asyncio.gather(*tasks)
                        tasks = []  # Reset the list of tasks after processing
                        await asyncio.sleep(1)  # Ensure we don't exceed 30 requests per second

                # Await any remaining tasks
                if tasks:
                    await asyncio.gather(*tasks)

                # Final progress update
                print(f"Final Progress: {progress['count']} summonerNames processed at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Example usage
source_csv_path = '../DataProcessing/TestData/GOLD_players.csv'
target_csv_path = '../DataProcessing/TestData/GOLD_GAMES.csv'
api_key = "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"

# Adapting to environments with an existing event loop
loop = asyncio.get_event_loop()

if loop.is_running():
    task = loop.create_task(run(source_csv_path, target_csv_path, api_key))
else:
    loop.run_until_complete(run(source_csv_path, target_csv_path, api_key))

    
    
    

Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400